The purpose of this notebook is to convert flat CSV data into hierarchical categorical data in JSON format. This is then passed to D3 to construct a sunburst chart. While we could do this slicing-and-dicing with D3 or other Javascript libraries, this provides a far more flexible implementation. By using Python to deal with complicated logic, Javascript chart code is kept uncluttered with data transformations.

We begin by importing Pandas, which we'll use to store the flat budget data. The defaultdict type provides a convenient data storage container, and the json library is used at the end when we dump out the processed hierarchical JSON to a JSON file.

In [2]:
import pandas as pd
from collections import defaultdict
import json

We start by defning a function to turn the flat dataframe data into a hierarchical JSON structure. This function will be called once for each year of budget data that we have. One call corresponds to one budget year corresponds to one sunburst chart.

In [3]:
def treeify_dataframe(df,groupby_groups,value_label):

    # Turn this flat dataframe into hierarchial, nested JSON
    #
    # groupby_groups:
    # variable labels that encode hierarchial information for the radian rings
    # Example: 'Fund name', 'Department', 'BCL Name'
    # 
    # value_label: 
    # values that are associated with keys, used for display stuff.
    # Example: 'Description', '2012 Amount'


    def rec_dd():
        return defaultdict(rec_dd)

    # First, create a recursive default dictionary
    # Populate it with our hierarchical structure
    tree = rec_dd()

    # the recursive default dictionary allows you to say something like,
    # my_rec_dd['a']['b']['c']['d']
    #
    # and when you print my_rec_dd, it will show as:
    # {'a': {'b': {'c': {'d': {}} } } }
    # 
    # This is a stepping stone to turning flat categorical data
    # into a nested hierarchy
    
    
    ###################
    # Group by groupby groups
    #
    bcl_groupby = df.groupby(groupby_groups)
    bcl_groups = bcl_groupby.groups
    

    
    ###################
    # JSON Tree Structure

    tree = rec_dd()
    
    # Populate the JSON skeleton
    
    for k,v in zip(bcl_groups.keys(),bcl_groups.values()):
        dfvalue = df.ix[v][value_label].values[0]
        
        if type(dfvalue)==type('asdf'):
            
            # in 2012 budget data, dollar values were strings, '$10000'
            tree[k[0]][k[1]][k[2]] = float(df.ix[v][value_label].values[0][1:])
        
        else:
            
            # in 2013/2014 budget data, dollar values were numbers, 10000
            tree[k[0]][k[1]][k[2]] = dfvalue

    
    
    ###################
    # D3 Tree Structure
    #
    # this looks like this:
    # 
    # { 'name' : 'node1',
    #   'children' : [ 
    #                  { 'name' : 'node2', 
    #   ...
    
    def treeify(node,curIndex):
        if curIndex==2:
            children = []
            for (k,v) in node.iteritems():
                child = {}
                child['name'] = k
                child['total'] = v
                children.append(child)
            return children
        else:
            children = []
            for k in node.keys():
                child = {}
                child['name'] = k
                child['children'] = treeify(node[k],curIndex+1)
                children.append(child)
            return children
    
    myjson = {}
    myjson['name'] = 'root'
    myjson['children'] = treeify(tree,0)
    
    return myjson


Now we call the function, once for each year. Start with 2012, then 2013, then 2014:

In [4]:
# Load 2012 budget data
df = pd.read_csv('budget2012.csv')

# these groupby groups form the rings of the sunburst chart
groupby_groups = ['Fund Name','Department','Budget Control Level (BCL) Name']

# this is the name of the dollar amount field
value_label ='2012 Expenditure Allowance'

# create treeified JSON structure
tree_json = treeify_dataframe(df,groupby_groups,value_label)

# and dump it to a JSON file
with open('budget2012.json','w') as f:
    json.dump(tree_json,f)

And just so we can see what the final JSON looks like:

In [7]:
print json.dumps(tree_json, separators=(',',': '), indent=4, sort_keys=False)

{
    "name": "root",
    "children": [
        {
            "name": "Special Employment Program Subfund (00515)",
            "children": [
                {
                    "name": "Personnel Department",
                    "children": [
                        {
                            "total": 200000.0,
                            "name": "Special Employment"
                        }
                    ]
                }
            ]
        },
        {
            "name": "Firefighters Pension Fund (60200)",
            "children": [
                {
                    "name": "Firefighters' Pension",
                    "children": [
                        {
                            "total": 19829267.0,
                            "name": "Firefighters' Pension"
                        }
                    ]
                }
            ]
        },
        {
            "name": "Streetcar Fund (10810)",
            "children": [
                {
         

Now finish the remaining two years:

In [8]:
df = pd.read_csv('budget2013.csv')
groupby_groups = ['Fund','Department','BCL Name']
value_label = '2013 Expenditure Allowance'
tree_json = treeify_dataframe(df,groupby_groups,value_label)

with open('budget2013.json','w') as f:
    json.dump(tree_json,f)

In [9]:
df = pd.read_csv('budget2014.csv')
groupby_groups = ['Fund','Department','BCL Name']
value_label = '2014 Expenditure Allowance'
tree_json = treeify_dataframe(df,groupby_groups,value_label)

with open('budget2014.json','w') as f:
    json.dump(tree_json,f)

And we're done! Now we have hierarchical JSON files that we can load into D3 and use to create sunburst charts.